<a href="https://colab.research.google.com/github/ailunguo/Test/blob/main/Tensorflow_Test/%E5%B1%82API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 层API

In [2]:
import tensorflow as tf
from tensorflow.keras import layers

In [3]:
layer = layers.Dense(32, activation='relu')
inputs = tf.random.uniform(shape=(10, 20))
outputs = layer(inputs)

In [6]:
# 层维护一个状态，当层在训练期间接受数据时更新，并存储在layer.weights
layer.weights

[<tf.Variable 'dense/kernel:0' shape=(20, 32) dtype=float32, numpy=
 array([[-0.05567047,  0.31292653, -0.13742629, -0.2273688 ,  0.26214993,
         -0.23777099, -0.04089954, -0.20524241,  0.30012488,  0.32383823,
          0.30036312, -0.02244112,  0.08285853,  0.18708074,  0.1158919 ,
         -0.2807782 , -0.2572527 , -0.19771299,  0.14625481,  0.30086982,
          0.08598122,  0.03751105, -0.13985816,  0.0642311 , -0.1418638 ,
         -0.33267656, -0.03955612,  0.3238871 , -0.03714079,  0.17921627,
         -0.04312432, -0.27468476],
        [ 0.32543623,  0.239528  , -0.00789338,  0.01627186,  0.26870483,
         -0.0443435 ,  0.29473317, -0.08163288, -0.32503843, -0.25181764,
          0.10047501,  0.19757944, -0.24360624,  0.10487357, -0.3163072 ,
          0.20873171,  0.28273028, -0.04418761,  0.0934256 , -0.26690453,
          0.19274998,  0.24786443,  0.3316276 ,  0.19501501,  0.08874011,
          0.16809136, -0.28583744, -0.25757354, -0.14694855, -0.0962991 ,
        

## 基础层类

In [ ]:
# Layer类
tf.keras.layers.Layer(
    trainable==True, name=None, dtype=None, dynamic=False, **kwargs
)
# dynamic：布尔值，False则生成静态计算图，True就是要急切的运行

In [14]:
# 实例
class SimpleDense(layers.Layer):

  def __init__(self, units=32):
    super(SimpleDense, self).__init__()
    self.units = units

  def build(self, input_shape):
    w_init = tf.random_normal_initializer()
    self.w = tf.Variable(
        initial_value=w_init(shape=(input_shape[-1], self.units),
                     dtype='float32'),
        trainable=True)
    b_init = tf.zeros_initializer()
    self.b = tf.Variable(
        initial_value=b_init(shape=(self.units,), dtype='float32'),
        trainable=False)

  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

linear_layer = SimpleDense(4)

y = linear_layer(tf.ones((2, 2)))
assert len(linear_layer.weights) == 2

assert len(linear_layer.trainable_weights) == 1

In [9]:
tf.random_normal_initializer()(shape=(2, 2), dtype='float32')

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-0.00342907, -0.00129643],
       [ 0.0324378 ,  0.05942843]], dtype=float32)>

In [16]:
# add_weight()方法提供了创建权重的方法
# 改进上述SimpleDense
class SimpleDense(layers.Layer):

  def __init__(self, units=32):
    super(SimpleDense, self).__init__()
    self.units = units

  def build(self, input_shape):
    self.w = self.add_weight(shape=(input_shape[-1], self.units),
                  initializer='random_normal',
                  trainable=True)
    self.b = self.add_weight(shape=(self.units,),
                  initializer='ramdom_normal',
                  trainable=True)

  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b


In [19]:
# 计算输入的运行总和的实例层
class ComputeSum(layers.Layer):

  def __init__(self, input_dim):
    super(ComputeSum, self).__init__()
    self.total = tf.Variable(initial_value=tf.zeros((input_dim,)),
                  trainable=False)

  def call(self, inputs):
    self.total.assign_add(tf.reduce_sum(inputs, axis=0))
    return self.total

In [17]:
tf.reduce_sum(tf.ones((2,2)), axis=0)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([2., 2.], dtype=float32)>

In [21]:
my_sum = ComputeSum(2)
x = tf.ones((2, 2))

y = my_sum(x)
print(y.numpy())

y = my_sum(x) # 继续累加，在权重上的累加
print(y.numpy())
print(my_sum.weights)
print(my_sum.total)
assert my_sum.weights == [my_sum.total]
assert my_sum.non_trainable_weights == [my_sum.total]
assert my_sum.trainable_weights == []

[2. 2.]
[4. 4.]
[<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([4., 4.], dtype=float32)>]
<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([4., 4.], dtype=float32)>
